In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os, sys
import pandas as pd
import PIL.Image
import scipy.io

root = '/usr/local/serenceslab/maggie/shapeDim/'

sys.path.append(os.path.join(root, 'Analysis'))
from code_utils import file_utils, data_utils, plot_utils

In [2]:
subjects = np.arange(1,8)
rois = ['V1','V2','V3','V3AB','hV4','IPS0','IPS1','IPS2','IPS3','LO1','LO2'];
n_rois = len(rois)
hemis = ['lh', 'rh']
rois_full = np.concatenate([['%s_%s'%(rr,hh) for hh in hemis] for rr in rois])
n_subjects = len(subjects)

raw_all = np.zeros((n_subjects, len(rois_full)))
thresh_all = np.zeros((n_subjects, len(rois_full)))

thresh_sum = np.zeros((n_subjects, len(rois)))

for si, ss in enumerate(subjects):

    # load this file, made by get_roi_sizes.m
    roi_size_file = os.path.join(root, 'Samples', 'S%02d_roi_sizes.mat'%ss)

    r = scipy.io.loadmat(roi_size_file)
    roi_sizes_raw = r['roi_sizes_raw']
    roi_sizes_thresh = r['roi_sizes_thresh']

    raw_all[si,:] = roi_sizes_raw.ravel()
    thresh_all[si,:] = roi_sizes_thresh.ravel()
    
    thresh_sum[si,:] = np.sum(roi_sizes_thresh, axis=1)

#### checking that these sizes match sizes of ROIs in the preprocessed data

In [43]:
for si, ss in enumerate(subjects):
    
    rep_data, _, rep_labels, roi_names = data_utils.load_repeat_task_data(ss, make_time_resolved = False)

    sizes = [rep_data[ri].shape[1] for ri in range(n_rois)]
    assert(np.all(sizes==thresh_sum[si,:]))

print('check successful')

missing voxel_inds for IPS2 lh
check successful


#### Final sizes (after thresholding and removing overlap)

In [47]:
thresh_df = pd.DataFrame(thresh_sum.astype(int).T, index=rois, columns=subjects)
thresh_df

,1,2,3,4,5,6,7
V1,1866,1956,2216,2464,2156,2373,1520
V2,1236,1834,1866,1584,1424,1570,1098
V3,1109,1929,1200,1548,1430,1228,1710
V3AB,1013,1964,517,708,812,1256,1171
hV4,277,641,484,1080,578,572,636
IPS0,476,640,182,548,537,478,622
IPS1,182,244,64,283,136,266,528
IPS2,85,107,5,189,211,325,332
IPS3,271,230,38,176,201,388,282
LO1,369,331,462,465,352,465,768


In [48]:
thresh_df = pd.DataFrame(thresh_all.astype(int).T, index=rois_full, columns=subjects)
thresh_df

,1,2,3,4,5,6,7
V1_lh,734,921,941,1155,1061,1285,746
V1_rh,1132,1035,1275,1309,1095,1088,774
V2_lh,588,1014,908,756,707,689,541
V2_rh,648,820,958,828,717,881,557
V3_lh,600,955,575,746,751,624,887
V3_rh,509,974,625,802,679,604,823
V3AB_lh,492,794,267,331,400,609,498
V3AB_rh,521,1170,250,377,412,647,673
hV4_lh,152,357,210,518,243,231,346
hV4_rh,125,284,274,562,335,341,290


#### Raw sizes

In [ ]:
raw_df = pd.DataFrame(raw_all.astype(int).T, index=rois_full, columns=subjects)
raw_df

,1,2,3,4,5,6,7
V1_lh,1007,1041,1222,1479,1654,1634,1195
V1_rh,1414,1263,1496,1706,1383,1297,1207
V2_lh,901,1386,1316,1389,1037,1111,998
V2_rh,950,1189,1362,1274,1014,1270,1226
V3_lh,872,1428,931,1133,1001,960,1413
V3_rh,661,1447,918,1245,880,938,1337
V3AB_lh,804,1065,474,556,664,699,755
V3AB_rh,689,1466,651,618,670,1138,996
hV4_lh,168,460,330,550,266,310,403
hV4_rh,133,310,306,590,363,541,390


#### Proportion passing threshold

In [52]:
prop_df = pd.DataFrame((thresh_all/raw_all).T.round(2), index=rois_full, columns=subjects)
prop_df

,1,2,3,4,5,6,7
V1_lh,0.73,0.88,0.77,0.78,0.64,0.79,0.62
V1_rh,0.80,0.82,0.85,0.77,0.79,0.84,0.64
V2_lh,0.65,0.73,0.69,0.54,0.68,0.62,0.54
V2_rh,0.68,0.69,0.70,0.65,0.71,0.69,0.45
V3_lh,0.69,0.67,0.62,0.66,0.75,0.65,0.63
V3_rh,0.77,0.67,0.68,0.64,0.77,0.64,0.62
V3AB_lh,0.61,0.75,0.56,0.60,0.60,0.87,0.66
V3AB_rh,0.76,0.80,0.38,0.61,0.61,0.57,0.68
hV4_lh,0.90,0.78,0.64,0.94,0.91,0.75,0.86
hV4_rh,0.94,0.92,0.90,0.95,0.92,0.63,0.74


#### Versions with bigger IPS (no thresholding)

In [2]:
subjects = np.arange(1,8)
rois = ['V1','V2','V3','V3AB','hV4','IPS0','IPS1','IPS2','IPS3','LO1','LO2'];
n_rois = len(rois)
hemis = ['lh', 'rh']
rois_full = np.concatenate([['%s_%s'%(rr,hh) for hh in hemis] for rr in rois])
n_subjects = len(subjects)

raw_all = np.zeros((n_subjects, len(rois_full)))
thresh_all = np.zeros((n_subjects, len(rois_full)))

thresh_sum = np.zeros((n_subjects, len(rois)))

for si, ss in enumerate(subjects):

    # load this file, made by get_roi_sizes.m
    roi_size_file = os.path.join(root, 'Samples', 'S%02d_roi_sizes_bigIPS.mat'%ss)

    r = scipy.io.loadmat(roi_size_file)
    roi_sizes_raw = r['roi_sizes_raw']
    roi_sizes_thresh = r['roi_sizes_thresh']

    raw_all[si,:] = roi_sizes_raw.ravel()
    thresh_all[si,:] = roi_sizes_thresh.ravel()
    
    thresh_sum[si,:] = np.sum(roi_sizes_thresh, axis=1)

In [3]:
roi_sizes_raw

array([[1195, 1207],
       [ 998, 1226],
       [1413, 1337],
       [ 755,  996],
       [ 403,  390],
       [ 431,  544],
       [ 417,  426],
       [ 188,  447],
       [ 258,  295],
       [ 429,  430],
       [ 259,  223]], dtype=uint16)

#### checking that these sizes match sizes of ROIs in the preprocessed data

In [4]:
for si, ss in enumerate(subjects):
    
    rep_data, _, rep_labels, roi_names = data_utils.load_repeat_task_data(ss, \
                                                                          make_time_resolved = False, \
                                                                          use_bigIPS=True)

    sizes = [rep_data[ri].shape[1] for ri in range(n_rois)]
    assert(np.all(sizes==thresh_sum[si,:]))

print('check successful')

check successful


#### Final sizes (after thresholding and removing overlap)

In [6]:
thresh_df = pd.DataFrame(thresh_sum.astype(int).T, index=rois, columns=subjects)
thresh_df

,1,2,3,4,5,6,7
V1,1866,1956,2216,2464,2156,2373,1520
V2,1236,1834,1866,1584,1424,1570,1098
V3,1109,1929,1200,1548,1430,1228,1710
V3AB,1013,1964,517,708,812,1256,1171
hV4,277,641,484,1080,578,572,636
IPS0,950,1080,690,783,913,739,888
IPS1,628,543,647,447,896,477,768
IPS2,789,362,477,464,956,409,577
IPS3,687,478,522,693,630,605,528
LO1,369,331,462,465,352,465,768


In [7]:
thresh_df = pd.DataFrame(thresh_all.astype(int).T, index=rois_full, columns=subjects)
thresh_df

,1,2,3,4,5,6,7
V1_lh,734,921,941,1155,1061,1285,746
V1_rh,1132,1035,1275,1309,1095,1088,774
V2_lh,588,1014,908,756,707,689,541
V2_rh,648,820,958,828,717,881,557
V3_lh,600,955,575,746,751,624,887
V3_rh,509,974,625,802,679,604,823
V3AB_lh,492,794,267,331,400,609,498
V3AB_rh,521,1170,250,377,412,647,673
hV4_lh,152,357,210,518,243,231,346
hV4_rh,125,284,274,562,335,341,290


#### Raw sizes

In [8]:
raw_df = pd.DataFrame(raw_all.astype(int).T, index=rois_full, columns=subjects)
raw_df

,1,2,3,4,5,6,7
V1_lh,1007,1041,1222,1479,1654,1634,1195
V1_rh,1414,1263,1496,1706,1383,1297,1207
V2_lh,901,1386,1316,1389,1037,1111,998
V2_rh,950,1189,1362,1274,1014,1270,1226
V3_lh,872,1428,931,1133,1001,960,1413
V3_rh,661,1447,918,1245,880,938,1337
V3AB_lh,804,1065,474,556,664,699,755
V3AB_rh,689,1466,651,618,670,1138,996
hV4_lh,168,460,330,550,266,310,403
hV4_rh,133,310,306,590,363,541,390


#### Proportion passing threshold

they are not exactly 1.0 for ips areas, this is because of double-counting, and having to 
remove voxels that overlapped (ips0/ips1 for example)

In [9]:
prop_df = pd.DataFrame((thresh_all/raw_all).T.round(2), index=rois_full, columns=subjects)
prop_df

,1,2,3,4,5,6,7
V1_lh,0.73,0.88,0.77,0.78,0.64,0.79,0.62
V1_rh,0.80,0.82,0.85,0.77,0.79,0.84,0.64
V2_lh,0.65,0.73,0.69,0.54,0.68,0.62,0.54
V2_rh,0.68,0.69,0.70,0.65,0.71,0.69,0.45
V3_lh,0.69,0.67,0.62,0.66,0.75,0.65,0.63
V3_rh,0.77,0.67,0.68,0.64,0.77,0.64,0.62
V3AB_lh,0.61,0.75,0.56,0.60,0.60,0.87,0.66
V3AB_rh,0.76,0.80,0.38,0.61,0.61,0.57,0.68
hV4_lh,0.90,0.78,0.64,0.94,0.91,0.75,0.86
hV4_rh,0.94,0.92,0.90,0.95,0.92,0.63,0.74
